In [12]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

import pycolmap
from deep_image_matching.triangulation import (
    create_db_from_model,
    import_keypoints,
    import_matches,
    import_verifed_matches,
)
from deep_image_matching.utils import (
    OutputCapture,
)

## Tringulate points with Pycolmap


In [13]:
# Define project directory
root_path = Path("sandbox/dense")

# Path to the images
image_dir = root_path / "images"

# Path to the pre-computed COLMAP reconstuction with the knwon poses
sfm_path = root_path / "sfm"
sfm_db_path = sfm_path / "database.db"
sfm_rec_path = sfm_path / "reconstruction"

# Path to the dense matching results to be triangulated
dense_path = root_path / "dense"
features_h5 = dense_path / "features.h5"
matches_h5 = dense_path / "matches.h5"
pair_file = dense_path / "pairs.txt"

# Path to the output for the dense matching
dense_db_path = dense_path / "database.db"
model_path = dense_path / "model"
model_path.mkdir(exist_ok=True)

In [14]:
# Import the sparse reconstruction
sfm_rec = pycolmap.Reconstruction(sfm_rec_path)

# Create an empty database from the sparse reconstruction
image_ids = create_db_from_model(sfm_rec, dense_db_path)

# Add keypoints to the database
import_keypoints(features_h5, image_ids, dense_db_path)

# Add matches to the database, but do not add two-view geometry
import_matches(
    matches_h5, image_ids, dense_db_path, pair_file, add_two_view_geometry=False
)

# Run the geometric verification with the knwon camera poses and add the inliers matches to the database in the two-view geometry table
import_verifed_matches(
    image_ids, sfm_rec, dense_db_path, features_h5, matches_h5, pair_file, max_error=4.0
)

Importing matches: 100%|██████████| 1/1 [00:00<00:00, 359.07it/s]
INFO:dim:Performing geometric verification of the matches...
Importing verified matches: 100%|██████████| 1/1 [00:00<00:00,  1.22it/s]
INFO:dim:mean/med/min/max valid matches 68.09/68.09/68.09/68.09%.


In [15]:
# Run the triangulation with the known camera poses

# Define the options for the triangulation according to the IncrementalPipelineOptions available in pycolmap
# print(pycolmap.IncrementalPipelineOptions().summary())
opt = dict(triangulation=dict(ignore_two_view_tracks=False))
verbose = True

with OutputCapture(verbose):
    with pycolmap.ostream():
        reconstruction = pycolmap.triangulate_points(
            sfm_rec,
            dense_db_path,
            image_dir,
            model_path,
            options=opt,
        )

I20240330 16:27:56.156109 4158779 misc.cc:198] 
Loading database
I20240330 16:27:56.157060 4158779 database_cache.cc:54] Loading cameras...
I20240330 16:27:56.157088 4158779 database_cache.cc:64]  2 in 0.000s
I20240330 16:27:56.157095 4158779 database_cache.cc:72] Loading matches...
I20240330 16:27:56.157989 4158779 database_cache.cc:78]  1 in 0.001s
I20240330 16:27:56.158003 4158779 database_cache.cc:94] Loading images...
I20240330 16:27:56.196110 4158779 database_cache.cc:143]  2 in 0.038s (connected 2)
I20240330 16:27:56.196182 4158779 database_cache.cc:154] Building correspondence graph...
I20240330 16:27:56.239976 4158779 database_cache.cc:190]  in 0.044s (ignored 0)
I20240330 16:27:56.240840 4158779 timer.cc:91] Elapsed time: 0.001 [minutes]
I20240330 16:27:56.259898 4158779 misc.cc:198] 
Triangulating image #1 (0)
I20240330 16:27:56.259933 4158779 sfm.cc:473] => Image sees 0 / 127384 points
I20240330 16:27:56.617060 4158779 sfm.cc:478] => Triangulated 127379 points
I20240330 16:

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  6.656609e+04    0.00e+00    1.48e+03   0.00e+00   0.00e+00  1.00e+04        0    9.13e-02    3.32e-01
   1  6.409387e+04    2.47e+03    1.46e-01   2.12e-01   1.00e+00  3.00e+04        0    1.09e-01    4.42e-01


I20240330 16:27:57.335784 4158779 misc.cc:205] 
Bundle adjustment report
------------------------
I20240330 16:27:57.335851 4158779 bundle_adjustment.cc:942] 
    Residuals : 509520
   Parameters : 382140
   Iterations : 2
         Time : 0.471104 [s]
 Initial cost : 0.361448 [px]
   Final cost : 0.354673 [px]
  Termination : Convergence

I20240330 16:27:57.367061 4158779 incremental_mapper.cc:175] => Completed observations: 0
I20240330 16:27:57.386754 4158779 incremental_mapper.cc:178] => Merged observations: 0
I20240330 16:27:57.429034 4158779 incremental_mapper.cc:160] => Filtered observations: 0
I20240330 16:27:57.429075 4158779 sfm.cc:521] => Changed observations: 0.000000
I20240330 16:27:57.546033 4158779 misc.cc:198] 
Extracting colors
